# Visualizing 3D Object Labels

Before starting this tutorial, you will need to have downloaded the `boreas-objects-v1` folder:

Follow these steps to download a sequence:
1. [Create an AWS account](https://aws.amazon.com/premiumsupport/knowledge-center/create-and-activate-aws-account/)
2. [Install the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html)
4. Use the AWS CLI to download the `boreas-objects-v1` bucket:
```
root=/path/to/data/boreas/
folder=boreas-objects-v1
aws s3 sync s3://boreas/$folder $root$folder
```

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.transforms as mtf
import numpy as np

from pyboreas import BoreasDataset
from pyboreas.data.splits import obj_train
from pyboreas.utils.utils import get_inverse_tf

In [ ]:
%matplotlib inline
bd = BoreasDataset('/workspace/raid/krb/boreas/', split=obj_train, labelFolder="labels_detection")
seq = bd.sequences[36]  ## ** Try different sequences!
seq.filter_frames_gt()
seq.synchronize_frames('lidar')
index = 0  ## ** Try different indices in the sequence!
cam = seq.get_camera(index)
lid = seq.get_lidar(index)
rad = seq.get_radar(index)

bounds = [-75, 75, -75, 75, -5, 10] # xmin, xmax, ymin, ymax, zmin, zmax
bbs = lid.get_bounding_boxes()
bbs.filter_empty()
bbs.passthrough(bounds)

T_enu_camera = cam.pose
T_enu_lidar = lid.pose
T_camera_lidar = np.matmul(get_inverse_tf(T_enu_camera), T_enu_lidar)

lid.passthrough(bounds)
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot()
rot = 90
M = mtf.Affine2D().rotate_deg(rot)+plt.gca().transData
ax.scatter(lid.points[:, 0], lid.points[:, 1], s=1, c=lid.points[:, 2], vmin=-5, vmax=10, transform=M)
ax.axis('equal')
bbs.render_2d(ax, transform=M)
plt.show()

In [ ]:
%matplotlib inline
lid.remove_motion(lid.body_rate, tref=lid.points[-1, -1])
lid.transform(T_camera_lidar)
lid.passthrough([-75, 75, -20, 20, 2, 40])
uv, colors, mask = lid.project_onto_image(seq.calib.P0)
fig = plt.figure(figsize=(24.48, 20.48), dpi=100)
ax = fig.add_subplot()
ax.imshow(cam.img)
ax.set_xlim(0, 2448)
ax.set_ylim(2048, 0)
ax.scatter(uv[:, 0], uv[:, 1], c=colors, marker=',', s=3, edgecolors='none', alpha=0.7, cmap='jet')
ax.set_axis_off()
plt.show()

In [ ]:
%matplotlib inline
# bbs.transform(T_camera_lidar)
bbs = cam.get_bounding_boxes(seq.labelFiles, seq.labelTimes, seq.labelPoses)
UV = bbs.project(seq.calib.P0)
bbs.visualize(cam.img, seq.calib.P0, checkdims=True)
cam.visualize()

In [ ]:
from pyboreas.utils.utils import get_T_bev_metric
resolution = 0.25
width = int(150 / resolution)
bbs = rad.get_bounding_boxes(seq.labelFiles, seq.labelTimes, seq.labelPoses)
bbs.passthrough(bounds)
T_bev_metric = get_T_bev_metric(resolution, width)
bbs.transform(T_bev_metric)
ax = rad.visualize(show=False, cart_resolution=resolution, cart_pixel_width=width)
bbs.render_2d(ax)
plt.show()